In [1]:
import numpy as np
import jax
import jax.numpy as jnp
from jax import vmap
from jax import jit, lax
import matplotlib.pyplot as plt
import cmocean as cmo
import importlib

In [2]:
from jax.config import config
config.update("jax_enable_x64", True)

In [161]:
## import modules
import preconditioner as precond
import conjugate_gradient as cg
import pivoted_cholesky as pc
import pivoted_cholesky_ref as pc_ref # to use this script we need "torch", please comment out if not needed.
import calc_logdet
def reload():
    importlib.reload(precond)
    importlib.reload(cg)
    importlib.reload(pc)
    importlib.reload(pc_ref)
    importlib.reload(calc_logdet)
reload()

In [162]:
import gpytorch
import torch
import linear_operator
from linear_operator.operators import (
    AddedDiagLinearOperator,
    DiagLinearOperator,
    LinearOperator,
    DenseLinearOperator,
)

In [163]:
def generate_K(N, seed=0, noise=1e-06):
    """
    generate positive definite symmetric matrix
    """
    K = jax.random.normal(jax.random.PRNGKey(seed), (N, N))
    # K = K @ K.T + 30* jnp.eye(N) + noise*jnp.eye(N)
    # K = jnp.dot(K, K.T) + noise*jnp.eye(N)
    # K = jnp.dot(K, K.T) / N
    K = jnp.dot(K, K.T)/N
    # K += (noise+30)*jnp.eye(N) ## ??
    K += (5)*jnp.eye(N)
    K += (noise)*jnp.eye(N)
    if not is_positive_definite(K):
        raise Exception("K is not positive definite !")
    return K

In [164]:
def is_positive_definite(matrix):
    # 行列の固有値を計算
    eigenvalues = np.linalg.eigvals(matrix)

    # 全ての固有値が正であるかをチェック
    if np.all(eigenvalues > 0):
        return True
    else:
        return False

In [165]:
def rel_error(true, pred):
    nonzero_index = jnp.where(true != 0.)
    true = true[nonzero_index]
    pred = pred[nonzero_index]
    return jnp.mean(jnp.abs((true-pred)/true))

## 5. modified preconditioned conjugate gradient

### N=5000

In [123]:
N = 5000
rank=15
noise = 1e-06
n_tridiag = 10
K = generate_K(N)
y = jax.random.normal(key=jax.random.PRNGKey(0), shape=(N,1))
zs = jax.random.normal(jax.random.PRNGKey(0), (N, n_tridiag))
preconditioner = precond.Preconditioner(K, rank=rank, noise=noise)
K_torch = torch.from_numpy(np.array(K))
rhs = jnp.concatenate([zs, y], axis=1)
rhs_torch = torch.from_numpy(np.array(rhs))

In [124]:
Kinvy_linalg = jnp.linalg.solve(K, rhs)

In [166]:
preconditioner = precond.Preconditioner(K, rank, noise)
Kinvy, t_mat = cg.mpcg_bbmm(K, rhs, preconditioner=preconditioner, print_process=True, tolerance=1, n_tridiag=n_tridiag)
print(rel_error(Kinvy_linalg, Kinvy))

j=0 r1norm: 0.05273127789083324
j=1 r1norm: 0.018229989448990253
j=2 r1norm: 0.016712777654460265
j=3 r1norm: 0.016678237420476163
j=4 r1norm: 0.01667748463554563
j=5 r1norm: 0.016677473148952585
j=6 r1norm: 0.016677456685424495
j=7 r1norm: 0.016677112063466847
j=8 r1norm: 0.016665606870972682
j=9 r1norm: 0.016125945002406064
j=10 r1norm: 0.008159012203917657
converged
0.20706069925691578


In [167]:
K_linear_op = linear_operator.to_linear_operator(K_torch)
diag_tensor = torch.ones(N, dtype=torch.float64)*noise
diag_linear_op = DiagLinearOperator(diag_tensor)
added_diag = AddedDiagLinearOperator(K_linear_op, diag_linear_op)

In [168]:
preconditioner_torch, _, precond_logdet_torch = added_diag._preconditioner()
Kinvy_torch, t_mat_torch = added_diag._solve(rhs_torch, preconditioner=preconditioner_torch, num_tridiag=n_tridiag)
print(rel_error(Kinvy_linalg, Kinvy_torch.numpy()))
linear_operator.settings.cg_tolerance._set_value(1)

3.5123671374860206e-07


In [169]:
%%timeit
preconditioner = precond.Preconditioner(K, rank, noise)
Kinvy = cg.mpcg_bbmm(K, rhs, preconditioner=preconditioner, print_process=False, tolerance=1, n_tridiag=n_tridiag)

297 ms ± 2.88 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [170]:
%%timeit
preconditioner_torch, _, _ = added_diag._preconditioner()
Kinvy_torch, t_mat = added_diag._solve(rhs_torch, preconditioner=preconditioner_torch, num_tridiag=n_tridiag)

1.09 s ± 3.68 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [171]:
%%timeit
Kinvy_linalg = jnp.linalg.solve(K, rhs)

306 ms ± 615 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [172]:
calc_logdet.calc_logdet(K.shape, t_mat, preconditioner)

DeviceArray(9108.50948291, dtype=float64)

In [173]:
added_diag.inv_quad_log_det(inv_quad_rhs=rhs_torch[:, -1:], logdet=True)

(tensor(869.8112, dtype=torch.float64), tensor(8847.2154, dtype=torch.float64))

In [174]:
eval_torch, evec_torch = linear_operator.utils.lanczos.lanczos_tridiag_to_diag(t_mat_torch)
slq = linear_operator.utils.stochastic_lq.StochasticLQ()
(logdet_term,) = slq.to_dense(added_diag.matrix_shape, eval_torch, evec_torch,  [lambda x: x.log()])

In [175]:
logdet_term + precond_logdet_torch

tensor(9127.5788, dtype=torch.float64)